In [ ]:
pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Tratamiento de los datos

In [ ]:
# Se descargan los datos semanales de META
db = yf.download('META', start = '2013-01-01', end = '2023-10-30', interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
db['EXMM9'] = db['Adj Close'].ewm(span = 9, adjust = False).mean()

 # Media movil simple a 20 dias
db['MM20'] = db['Adj Close'].rolling(window=20).mean()

In [ ]:
# Definicion de parametros para el cruce de medias moviles
short_ma = 'EXMM9'
long_ma = 'MM20'

# Se crea un a señal que almacena las señales
db['señal'] = 0

# Se generan las señales de entrada
for i in range(1, len(db)):
    if db[short_ma][i] > db[long_ma][i] and db[short_ma][i - 1] <= db[long_ma][i - 1]:
# Señal de compra
        db['señal'][i] = 1
    elif db[short_ma][i] < db[long_ma][i] and db[short_ma][i - 1] >= db[long_ma][i - 1]:
       # Señal de venta
        db['señal'][i] = -1

<ipython-input-5-19f5fae6ba52>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['señal'][i] = -1
<ipython-input-5-19f5fae6ba52>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['señal'][i] = 1


#Visualizacion de los datos

## Creacion de un grafico para mostrar el periodo de trabajo

In [ ]:
# Crear gráfico
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.6])

# Candlestick
fig.add_trace(go.Candlestick(x=db.index,
                             open=db['Open'],
                             high=db['High'],
                             low=db['Low'],
                             close=db['Adj Close'],
                             name='META'),
              row=1, col=1)

# Añadir flechas al gráfico
sell_arrows = db[db['señal'] == -1]
buy_arrows = db[db['señal'] == 1]

fig.add_trace(go.Scatter(x=sell_arrows.index, y=sell_arrows['High'] + 10,
                         mode='markers', marker_symbol='triangle-down',
                         marker=dict(color='red', size=10),
                         name='Vender', legendgroup='sell'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=buy_arrows.index, y=buy_arrows['Low'] - 10,
                         mode='markers', marker_symbol='triangle-up',
                         marker=dict(color='green', size=10),
                         name='Comprar', legendgroup='buy'),
              row=1, col=1)

# Personalizar diseño
fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
fig.update_layout(template='plotly_dark')  # Usar el tema oscuro

# Mostrar el gráfico
fig.show()

#Preparacion de los datos

##Se generan los directorios donde se guardaran las imagenes

In [ ]:
import os
# Ruta del directorio META
meta_directory = 'META'

# Verificar si el directorio META existe, si no lo crea
if not os.path.exists(meta_directory):
    os.makedirs(meta_directory)

# Lista de directorios a crear dentro de META
sub_directories = ['comprar', 'vender', 'mantener']

# Crear los directorios dentro del directorio META
for directory in sub_directories:
    directory_path = os.path.join(meta_directory, directory)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

##Generaremos imagenes con informacion por dia (se pueden elegir distintos agrupamientos)

In [ ]:
# 'Date' a formato datetime
db.index = pd.to_datetime(db.index)

# Definir la duración de cada segmento (1 día en este caso), que corresponde a 1 imagen
dias_por_segmento = 1

# Divide los datos en segmentos de 1 día usando el índice
segmentos = [db.loc[start_date:start_date + pd.DateOffset(days=dias_por_segmento - 1)]
             for start_date in pd.date_range(start=db.index[0], end=db.index[-1], freq=f'{dias_por_segmento}D')]

In [ ]:
# Itera sobre los segmentos y crea un gráfico para cada uno
for segmento in segmentos:
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.6])

    # Añadir datos al gráfico
    fig.add_trace(go.Candlestick(x=segmento.index,
                                 open=segmento['Open'],
                                 high=segmento['High'],
                                 low=segmento['Low'],
                                 close=segmento['Adj Close'],
                                 name='META'),
                  row=1, col=1)


    # Se personaliza el diseño
    fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
    fig.update_layout(template='plotly_dark')

    # Se Formatea el eje x con día, mes y año
    fig.update_xaxes(type='category')
    fig.update_xaxes(tickvals=segmento.index, ticktext=segmento.index.strftime('%d-%m-%Y'), row=1, col=1)

    # Se verifican condiciones sobre la columna 'señal' y segun su valor guardar el gráfico en la carpeta adecuada

    if segmento['señal'].eq(1).any():
     fecha = segmento.index[0].strftime('%Y-%m-%d')
     fig_file_path = f"META/vender/{fecha}.png"
     pio.write_image(fig, fig_file_path)
    elif segmento['señal'].eq(-1).any():
     fecha = segmento.index[0].strftime('%Y-%m-%d')
     fig_file_path = f"META/comprar/{fecha}.png"
     pio.write_image(fig, fig_file_path)
    elif segmento['señal'].eq(0).any():
     fecha = segmento.index[0].strftime('%Y-%m-%d')
     fig_file_path = f"META/mantener/{fecha}.png"
     pio.write_image(fig, fig_file_path)

## Preparacion de las imagenes para la CNN

In [ ]:
import tensorflow as tf

In [ ]:
batch_size = 128

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    zoom_range = [0.8, 1.2],
    horizontal_flip=True
)



train_generator = train_datagen.flow_from_directory(
    '/content/META',  # Directorio donde están las imágenes
    target_size=(100, 100),
    batch_size=batch_size,
    classes=['comprar', 'mantener', 'vender']
)

Found 2725 images belonging to 3 classes.


# Diseño y entrenamiento de la red neuronal

##Definimos f1_score ya que no esta en Keras

In [ ]:
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())

    return K.mean(f1)

##Arquitectura de la CNN y entrenamiento

In [ ]:
model = tf.keras.models.Sequential([

    # Entrada
    tf.keras.layers.Input(shape=(100, 100, 3)), # 3 en color, 1 es grayscale

    # Convolución 1
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Convolución 2
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Convolución 3
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Convolución 4
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Convolución 5
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Achatamiento
    tf.keras.layers.Flatten(),

    # Capas densas
    tf.keras.layers.Dense(100, activation='relu'),

    # cantidad de neuronas de salida, una por clase
    tf.keras.layers.Dense(3, activation='softmax')

])

# Compila el modelo usando la métrica personalizada del F1-Score
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1_score])

n_epochs = 30
history = model.fit(train_generator, epochs=n_epochs, verbose=2)
